# Chapter 17: 封裝與資訊隱藏 | Encapsulation and Information Hiding

---

## Part I: 理論基礎

### 📚 章節概覽

**學習時數**：60 分鐘  
**難度等級**：⭐⭐⭐☆☆  
**先備知識**：Chapter 16（類別與物件）

---

### 🎯 本章學習地圖

```
封裝與資訊隱藏
│
├─ 為什麼需要封裝？
│  ├─ 保護資料完整性
│  ├─ 控制存取方式
│  └─ 隱藏實作細節
│
├─ Python 的存取控制約定
│  ├─ Public（公開）- name
│  ├─ Protected（保護）- _name
│  └─ Private（私有）- __name
│
├─ Name Mangling（名稱修飾）
│  └─ __name → _ClassName__name
│
├─ Property 裝飾器
│  ├─ @property（getter）
│  ├─ @name.setter（setter）
│  └─ @name.deleter（deleter）
│
└─ 封裝設計模式
   ├─ Read-only properties
   ├─ Computed properties
   └─ Validated properties
```

---

### 🔑 核心概念解析

#### 1. 為什麼需要封裝？（First Principles）

**根本問題**：如何防止外部代碼直接修改物件內部狀態，導致物件進入無效狀態？

**情境演示**：

In [ ]:
# ❌ 沒有封裝的問題：銀行帳戶餘額可以被隨意修改
class BankAccount_Bad:
    def __init__(self, balance):
        self.balance = balance  # 公開屬性

account = BankAccount_Bad(1000)
print(f"初始餘額: {account.balance}")

# ❌ 可以直接設定為負數！這不合理
account.balance = -500
print(f"修改後餘額: {account.balance}")  # -500（這是錯誤的狀態）

In [ ]:
# ✅ 使用封裝解決問題
class BankAccount_Good:
    def __init__(self, balance):
        self.__balance = balance  # 私有屬性（雙底線）

    def deposit(self, amount):
        """存款（受控的操作）"""
        if amount > 0:
            self.__balance += amount
            print(f"存款 {amount}，目前餘額: {self.__balance}")
        else:
            print("存款金額必須為正數")

    def withdraw(self, amount):
        """提款（有驗證邏輯）"""
        if amount <= 0:
            print("提款金額必須為正數")
        elif amount > self.__balance:
            print("餘額不足")
        else:
            self.__balance -= amount
            print(f"提款 {amount}，目前餘額: {self.__balance}")

    def get_balance(self):
        """取得餘額（唯讀）"""
        return self.__balance

# 測試
account = BankAccount_Good(1000)
print(f"初始餘額: {account.get_balance()}")

account.deposit(500)
account.withdraw(200)
account.withdraw(2000)  # 餘額不足，無法提款

# ✅ 無法直接修改餘額（會報錯）
try:
    account.__balance = -500
    print(f"餘額: {account.__balance}")  # 這行不會執行
except AttributeError as e:
    print(f"錯誤: {e}")

**封裝的優勢**：
- ✅ 防止無效資料（餘額不能為負數）
- ✅ 集中控制邏輯（所有操作都經過驗證）
- ✅ 隱藏實作細節（外部不需知道內部如何儲存資料）

---

## Part II: 實作演練

### 範例 1：Python 的三種存取控制約定

In [ ]:
class AccessControl:
    """演示 Python 的三種存取控制約定"""
    
    def __init__(self):
        self.public = "公開屬性"        # Public（無前綴）
        self._protected = "保護屬性"    # Protected（單底線）
        self.__private = "私有屬性"     # Private（雙底線）

    def show_all(self):
        """內部方法可以存取所有屬性"""
        print(f"Public: {self.public}")
        print(f"Protected: {self._protected}")
        print(f"Private: {self.__private}")

obj = AccessControl()

# 1. Public 屬性：自由存取
print(obj.public)  # ✅ 可以存取
obj.public = "新的公開值"
print(obj.public)

print("\n" + "="*50 + "\n")

# 2. Protected 屬性：約定為內部使用，但仍可存取
print(obj._protected)  # ⚠️ 可以存取，但不建議（慣例上是內部使用）
obj._protected = "修改保護屬性"  # ⚠️ 可以修改，但不建議
print(obj._protected)

print("\n" + "="*50 + "\n")

# 3. Private 屬性：觸發 name mangling
try:
    print(obj.__private)  # ❌ AttributeError
except AttributeError:
    print("❌ 無法直接存取 __private")

# Name Mangling：Python 將 __private 改名為 _ClassName__private
print(f"實際名稱: {obj._AccessControl__private}")  # ⚠️ 仍可存取，但極不建議

print("\n" + "="*50 + "\n")
obj.show_all()  # 內部方法可存取所有屬性

**重要提醒**：
- Python 沒有真正的私有化（與 Java/C++ 不同）
- 這些是**命名慣例**，依賴開發者自律
- `__name` 觸發 name mangling，但仍可透過 `_ClassName__name` 存取

---

### 範例 2：Name Mangling 的運作機制

In [ ]:
class Parent:
    def __init__(self):
        self.__private = "parent secret"  # 私有屬性
        self._protected = "parent protected"

class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__private = "child secret"  # 不會覆寫父類別的 __private
        self._protected = "child protected"  # 會覆寫父類別的 _protected

c = Child()

# Name Mangling 的效果
print("=== Name Mangling 演示 ===")
print(f"父類別的私有屬性: {c._Parent__private}")  # "parent secret"
print(f"子類別的私有屬性: {c._Child__private}")   # "child secret"

print("\n=== Protected 屬性（無 Name Mangling）===")
print(f"Protected 屬性: {c._protected}")  # "child protected"（被覆寫）

# 檢查所有屬性
print("\n=== 物件的所有屬性 ===")
attrs = [attr for attr in dir(c) if not attr.startswith('_Child__') and not attr.startswith('_Parent__')]
print("一般屬性:", attrs[:5])  # 只顯示部分

private_attrs = [attr for attr in dir(c) if attr.startswith('_') and '__' in attr]
print("私有屬性（經過 mangling）:", private_attrs)

**Name Mangling 的目的**：
- 避免子類別意外覆寫父類別的私有屬性
- `__private` → `_ClassName__private`
- 仍可透過修飾後的名稱存取（但不應該這麼做）

---

### 範例 3：使用 @property 建立受控的屬性存取

In [ ]:
class Person:
    """使用 @property 建立受控的年齡屬性"""
    
    def __init__(self, name, age):
        self._name = name
        self._age = age  # 使用單底線（內部屬性）

    @property
    def age(self):
        """Getter：取得年齡"""
        print("[Getter] 取得年齡")
        return self._age

    @age.setter
    def age(self, value):
        """Setter：設定年齡（有驗證）"""
        print(f"[Setter] 設定年齡為 {value}")
        if not isinstance(value, int):
            raise TypeError("年齡必須是整數")
        if value < 0 or value > 150:
            raise ValueError("年齡必須在 0-150 之間")
        self._age = value

    @age.deleter
    def age(self):
        """Deleter：刪除年齡屬性"""
        print("[Deleter] 刪除年齡屬性")
        del self._age

# 測試
p = Person("Alice", 25)

# 像屬性一樣存取，實際呼叫 getter
print(f"年齡: {p.age}")

print("\n" + "="*50 + "\n")

# 像屬性一樣設定，實際呼叫 setter（有驗證）
p.age = 30
print(f"新年齡: {p.age}")

print("\n" + "="*50 + "\n")

# 驗證失敗的情況
try:
    p.age = -10  # ValueError
except ValueError as e:
    print(f"❌ 錯誤: {e}")

try:
    p.age = "25"  # TypeError
except TypeError as e:
    print(f"❌ 錯誤: {e}")

print("\n" + "="*50 + "\n")

# 刪除屬性
del p.age

**@property 的優勢**：
- ✅ 語法優雅（像屬性一樣使用）
- ✅ 可加入驗證邏輯
- ✅ 隱藏內部實作
- ✅ 易於重構（從公開屬性改為 property）

---

### 範例 4：唯讀屬性（Read-only Property）

In [ ]:
import math

class Circle:
    """圓形類別，面積和周長為唯讀屬性"""
    
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """半徑（可讀寫）"""
        return self._radius

    @radius.setter
    def radius(self, value):
        if value <= 0:
            raise ValueError("半徑必須為正數")
        self._radius = value

    @property
    def area(self):
        """面積（唯讀，無 setter）"""
        return math.pi * self._radius ** 2

    @property
    def circumference(self):
        """周長（唯讀，無 setter）"""
        return 2 * math.pi * self._radius

# 測試
c = Circle(5)
print(f"半徑: {c.radius}")
print(f"面積: {c.area:.2f}")
print(f"周長: {c.circumference:.2f}")

print("\n" + "="*50 + "\n")

# 修改半徑，面積和周長自動更新
c.radius = 10
print(f"新半徑: {c.radius}")
print(f"新面積: {c.area:.2f}")
print(f"新周長: {c.circumference:.2f}")

print("\n" + "="*50 + "\n")

# ❌ 無法直接設定面積（唯讀屬性）
try:
    c.area = 100
except AttributeError as e:
    print(f"❌ 錯誤: can't set attribute (area 是唯讀屬性)")

**唯讀屬性的應用場景**：
- 計算屬性（如面積、周長）
- 衍生屬性（從其他屬性計算而來）
- 不應被外部修改的屬性

---

### 範例 5：計算屬性（Computed Property）

In [ ]:
class Rectangle:
    """矩形類別，面積為計算屬性"""
    
    def __init__(self, width, height):
        self._width = width
        self._height = height

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        if value <= 0:
            raise ValueError("寬度必須為正數")
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        if value <= 0:
            raise ValueError("高度必須為正數")
        self._height = value

    @property
    def area(self):
        """面積（每次存取時重新計算）"""
        print("[計算中] 計算面積...")
        return self._width * self._height

    @property
    def perimeter(self):
        """周長（每次存取時重新計算）"""
        print("[計算中] 計算周長...")
        return 2 * (self._width + self._height)

# 測試
r = Rectangle(4, 5)
print(f"寬度: {r.width}, 高度: {r.height}")
print(f"面積: {r.area}")
print(f"周長: {r.perimeter}")

print("\n" + "="*50 + "\n")

# 修改寬度
r.width = 6
print(f"新寬度: {r.width}")
print(f"新面積: {r.area}")  # 自動重新計算
print(f"新周長: {r.perimeter}")  # 自動重新計算

**計算屬性的特點**：
- 不儲存值，每次存取時重新計算
- 自動反映依賴屬性的變化
- 適合不需要快取的輕量計算

---

### 範例 6：驗證屬性（Validated Property）

In [ ]:
class User:
    """使用者類別，具有多種驗證屬性"""
    
    def __init__(self, username, email, age):
        self.username = username  # 透過 setter 驗證
        self.email = email
        self.age = age

    @property
    def username(self):
        return self._username

    @username.setter
    def username(self, value):
        """使用者名稱驗證"""
        if not isinstance(value, str):
            raise TypeError("使用者名稱必須是字串")
        if len(value) < 3:
            raise ValueError("使用者名稱至少 3 個字元")
        if not value.isalnum():
            raise ValueError("使用者名稱只能包含字母和數字")
        self._username = value

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, value):
        """Email 驗證"""
        if not isinstance(value, str):
            raise TypeError("Email 必須是字串")
        if "@" not in value or "." not in value:
            raise ValueError("Email 格式不正確")
        self._email = value

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        """年齡驗證"""
        if not isinstance(value, int):
            raise TypeError("年齡必須是整數")
        if value < 0 or value > 150:
            raise ValueError("年齡必須在 0-150 之間")
        self._age = value

# 測試成功案例
user = User("alice123", "alice@example.com", 25)
print(f"使用者: {user.username}")
print(f"Email: {user.email}")
print(f"年齡: {user.age}")

print("\n" + "="*50 + "\n")

# 測試驗證失敗
test_cases = [
    ("username", "ab", "使用者名稱至少 3 個字元"),
    ("username", "alice@123", "使用者名稱只能包含字母和數字"),
    ("email", "invalid-email", "Email 格式不正確"),
    ("age", -5, "年齡必須在 0-150 之間"),
    ("age", 200, "年齡必須在 0-150 之間"),
]

for attr, value, expected_error in test_cases:
    try:
        setattr(user, attr, value)
    except (ValueError, TypeError) as e:
        print(f"❌ {attr} = {value!r}: {e}")

**驗證屬性的優勢**：
- ✅ 確保資料完整性
- ✅ 集中驗證邏輯
- ✅ 提供清晰的錯誤訊息
- ✅ 防止無效狀態

---

### 範例 7：密碼加密（實務應用）

In [ ]:
import hashlib

class SecureUser:
    """具有密碼加密功能的使用者類別"""
    
    def __init__(self, username, password):
        self._username = username
        self._password_hash = None
        self.password = password  # 透過 setter 加密

    @property
    def password(self):
        """密碼（只能寫，不能讀）"""
        raise AttributeError("密碼無法讀取（安全考量）")

    @password.setter
    def password(self, value):
        """設定密碼（自動加密）"""
        if len(value) < 6:
            raise ValueError("密碼至少 6 個字元")
        # 使用 SHA-256 加密
        self._password_hash = hashlib.sha256(value.encode()).hexdigest()
        print(f"✅ 密碼已加密儲存")

    def verify_password(self, password):
        """驗證密碼"""
        password_hash = hashlib.sha256(password.encode()).hexdigest()
        return password_hash == self._password_hash

    def get_info(self):
        """顯示使用者資訊（不包含密碼）"""
        return f"使用者: {self._username}, 密碼雜湊: {self._password_hash[:16]}..."

# 測試
user = SecureUser("alice", "secret123")
print(user.get_info())

print("\n" + "="*50 + "\n")

# ❌ 無法讀取密碼
try:
    print(user.password)
except AttributeError as e:
    print(f"❌ {e}")

print("\n" + "="*50 + "\n")

# ✅ 可以驗證密碼
print(f"密碼正確: {user.verify_password('secret123')}")
print(f"密碼錯誤: {user.verify_password('wrong')}")

print("\n" + "="*50 + "\n")

# ✅ 可以修改密碼
user.password = "newsecret456"
print(user.get_info())
print(f"新密碼驗證: {user.verify_password('newsecret456')}")

**安全設計模式**：
- ✅ 密碼只能寫入，無法讀取
- ✅ 自動加密儲存（SHA-256）
- ✅ 提供驗證方法
- ✅ 隱藏敏感資訊

---

### 範例 8：綜合應用 - 產品管理系統

In [ ]:
from datetime import datetime

class Product:
    """產品類別，具有完整的封裝設計"""
    
    # 類別屬性：所有產品共享
    _id_counter = 1000
    tax_rate = 0.05  # 稅率 5%

    def __init__(self, name, price, stock):
        self.__id = Product._id_counter  # 私有屬性：產品 ID
        Product._id_counter += 1
        
        self._name = name
        self._price = price
        self._stock = stock
        self._created_at = datetime.now()

    @property
    def id(self):
        """產品 ID（唯讀）"""
        return self.__id

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not value or not value.strip():
            raise ValueError("產品名稱不能為空")
        self._name = value.strip()

    @property
    def price(self):
        return self._price

    @price.setter
    def price(self, value):
        if value < 0:
            raise ValueError("價格不能為負數")
        self._price = value

    @property
    def stock(self):
        return self._stock

    @stock.setter
    def stock(self, value):
        if value < 0:
            raise ValueError("庫存不能為負數")
        self._stock = value

    @property
    def price_with_tax(self):
        """含稅價格（計算屬性）"""
        return self._price * (1 + Product.tax_rate)

    @property
    def is_in_stock(self):
        """是否有庫存（計算屬性）"""
        return self._stock > 0

    @property
    def total_value(self):
        """庫存總價值（計算屬性）"""
        return self._price * self._stock

    def restock(self, quantity):
        """補貨"""
        if quantity <= 0:
            raise ValueError("補貨數量必須為正數")
        self._stock += quantity
        print(f"✅ 補貨 {quantity} 件，目前庫存: {self._stock}")

    def sell(self, quantity):
        """銷售"""
        if quantity <= 0:
            raise ValueError("銷售數量必須為正數")
        if quantity > self._stock:
            raise ValueError(f"庫存不足（目前庫存: {self._stock}）")
        self._stock -= quantity
        print(f"✅ 銷售 {quantity} 件，目前庫存: {self._stock}")

    def __str__(self):
        return f"Product(ID={self.id}, 名稱={self.name}, 價格=${self.price}, 庫存={self.stock})"

# 測試
print("=== 創建產品 ===")
p1 = Product("iPhone 15", 30000, 50)
p2 = Product("MacBook Pro", 60000, 20)

print(p1)
print(p2)

print("\n=== 產品資訊 ===")
print(f"ID: {p1.id}（唯讀）")
print(f"名稱: {p1.name}")
print(f"原價: ${p1.price}")
print(f"含稅價: ${p1.price_with_tax:.2f}")
print(f"庫存: {p1.stock}")
print(f"是否有貨: {p1.is_in_stock}")
print(f"庫存總價值: ${p1.total_value:,}")

print("\n=== 修改價格 ===")
p1.price = 28000
print(f"新價格: ${p1.price}")
print(f"新含稅價: ${p1.price_with_tax:.2f}")

print("\n=== 銷售與補貨 ===")
p1.sell(10)
p1.restock(20)

print("\n=== 驗證失敗 ===")
try:
    p1.price = -100
except ValueError as e:
    print(f"❌ {e}")

try:
    p1.sell(100)  # 庫存不足
except ValueError as e:
    print(f"❌ {e}")

try:
    p1.id = 9999  # ID 唯讀
except AttributeError:
    print(f"❌ ID 是唯讀屬性，無法修改")

**綜合設計重點**：
1. **私有屬性**：`__id`（避免外部修改）
2. **保護屬性**：`_name`, `_price`, `_stock`（內部使用）
3. **唯讀屬性**：`id`（只有 getter）
4. **驗證屬性**：`name`, `price`, `stock`（setter 有驗證）
5. **計算屬性**：`price_with_tax`, `total_value`（動態計算）
6. **類別屬性**：`tax_rate`（所有產品共享）

---

## Part III: 本章總結

### 📌 知識回顧

#### 1. 封裝的核心價值
- **保護資料完整性**：防止無效狀態
- **控制存取方式**：集中驗證邏輯
- **隱藏實作細節**：外部不需知道內部如何實作
- **易於維護**：修改內部實作不影響外部使用

#### 2. Python 的存取控制約定

| 慣例 | 語法 | 意義 | 使用時機 |
|:-----|:-----|:-----|:---------|
| Public | `name` | 公開屬性 | 外部應該存取的屬性 |
| Protected | `_name` | 保護屬性 | 約定為內部使用（90% 情況） |
| Private | `__name` | 私有屬性 | 避免繼承衝突（10% 情況） |

#### 3. Property 裝飾器

```python
@property
def attr(self):          # Getter
    return self._attr

@attr.setter
def attr(self, value):   # Setter
    self._attr = value

@attr.deleter
def attr(self):          # Deleter
    del self._attr
```

#### 4. 封裝設計模式

| 模式 | 特點 | 範例 |
|:-----|:-----|:-----|
| 唯讀屬性 | 只有 getter，無 setter | `circle.area` |
| 計算屬性 | 每次存取時重新計算 | `rectangle.perimeter` |
| 驗證屬性 | Setter 有驗證邏輯 | `person.age` |
| 只寫屬性 | 只有 setter，無 getter | `user.password` |

---

### ⚠️ 常見誤區

#### 誤區 1：過度封裝
```python
# ❌ 過度封裝：簡單的資料類別不需要 property
class Point:
    @property
    def x(self):
        return self._x

# ✅ 簡單資料直接公開
class Point:
    def __init__(self, x, y):
        self.x = x  # 公開屬性即可
        self.y = y
```

#### 誤區 2：混淆 `_name` 與 `__name`
```python
# 預設使用單底線（90% 的情況）
self._internal = value

# 只在需要避免繼承衝突時使用雙底線
self.__really_private = value
```

#### 誤區 3：忘記 setter 的驗證
```python
# ❌ 有 getter 但 setter 無驗證
@property
def age(self):
    return self._age

@age.setter
def age(self, value):
    self._age = value  # 無驗證！

# ✅ Setter 應加入驗證
@age.setter
def age(self, value):
    if value < 0:
        raise ValueError("年齡不能為負數")
    self._age = value
```

---

### ✅ 自我檢核

完成本講義後，您應該能夠：
- [ ] 解釋封裝的必要性
- [ ] 區分 public、protected、private 屬性
- [ ] 使用 `@property` 建立受控屬性
- [ ] 設計唯讀、計算、驗證屬性
- [ ] 理解 name mangling 的運作機制
- [ ] 判斷何時應該封裝，何時不需要

---

### 🔗 延伸閱讀

**Python 官方文件**：
- [Property](https://docs.python.org/3/library/functions.html#property)
- [Private Variables](https://docs.python.org/3/tutorial/classes.html#private-variables)

**推薦主題**：
- Chapter 18：繼承與多型（封裝在繼承中的應用）
- Chapter 19：特殊方法（進階類別設計）
- Descriptors（更底層的屬性控制）

---

**練習建議**：完成本講義後，請依序進行：
1. `02-worked-examples.ipynb`（詳解範例）
2. `03-practice.ipynb`（課堂練習）
3. `04-exercises.ipynb`（課後習題）

**學習提醒**：封裝是 OOP 的核心原則，掌握封裝能讓您設計出更健壯、易維護的類別！